# 부도기업예측을 위한 재무변수 전처리<br>

1. 거래소번호 수 6자리 맞추기 -> 0으로 맞춰도 새로 CVS read시 0 다 누락되어 처리하지않기로 함 <br>
2. nan값 해결해야합니당 -> 결측치가 많은 칼럼은 중요도를 따진 후 feature selection 전 드랍 예정<br>
3. KIND에서 가져온 공시정보다 매칭되지 않은 13개 기업 공시제목 강제할당 처리 완 


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px
import tqdm

In [2]:
train_full = pd.read_csv(r'C:\apps\Ubion-CorporateBankruptcyAnalysis\Data\코드_재무변수.csv', encoding='euc-kr')
train = pd.read_csv(r'C:\apps\Ubion-CorporateBankruptcyAnalysis\Data\코드_재무변수.csv', encoding='euc-kr', usecols = ['회사명','거래소코드','회계년도','소속코드','상장폐지일'])
#bankrupt = pd.read_csv(r'C:\Users\user\Desktop\code\부실기업\kind_bankrupt_df.csv', index_col=0)

In [3]:
#회계년도 추출
train['년'] = train['회계년도'].str[:4]
train_full['년'] = train_full['회계년도'].str[:4]

#회계년도+1 칼럼 생성 : 사업보고서 연도와 공시 데이터 연도 시점차이로 인해 +1
train['년+1'] = train['회계년도'].str[0:4].astype(int) + 1
train_full['년+1'] = train_full['회계년도'].str[0:4].astype(int) + 1

### KIND에서 추출한 상폐관련 공시 리스트 CSV화

In [6]:
d0002 = pd.read_csv(r'C:\apps\Ubion-CorporateBankruptcyAnalysis\Data\부도사유데이터\부도사유0002.csv')
d0305 = pd.read_csv(r'C:\apps\Ubion-CorporateBankruptcyAnalysis\Data\부도사유데이터\부도사유0305.csv')
d0608 = pd.read_csv(r'C:\apps\Ubion-CorporateBankruptcyAnalysis\Data\부도사유데이터\부도사유0608.csv')
d0911 = pd.read_csv(r'C:\apps\Ubion-CorporateBankruptcyAnalysis\Data\부도사유데이터\부도사유0911.csv')
d1214 = pd.read_csv(r'C:\apps\Ubion-CorporateBankruptcyAnalysis\Data\부도사유데이터\부도사유1214.csv')
d1517 = pd.read_csv(r'C:\apps\Ubion-CorporateBankruptcyAnalysis\Data\부도사유데이터\부도사유1517.csv')
d1820 = pd.read_csv(r'C:\apps\Ubion-CorporateBankruptcyAnalysis\Data\부도사유데이터\부도사유1820.csv')

In [15]:
bankrupt_df = pd.concat([d0002, d0305,d0608, d0911, d1214, d1517, d1820], axis=0).reset_index()
bankrupt_df.drop(columns=['index'], inplace=True)

In [25]:
bankrupt_df

,시간,회사명,종목코드,공시제목,제출인
0,2002-10-31 18:27,정산애강,22220,부도발생및은행거래정지,정산애강
1,2002-10-08 12:06,대호특수강,21040,은행거래재개,대호특수강
2,2002-05-21 18:16,에이치엘비,28300,회사정리절차종결결정,에이치엘비
3,2002-04-17 17:42,에이치엘비,28300,회사정리절차종결신청,에이치엘비
4,2001-12-26 14:58,대호특수강,21040,회사정리절차개시결정,대호특수강
...,...,...,...,...,...
389,2020-02-12 16:21,에스앤더블류,103230,내부결산시점 관리종목 지정 또는 상장폐지 사유 발생,에스앤더블류
390,2020-02-11 17:02,파나진,46210,내부결산시점 관리종목 지정 또는 상장폐지 사유 발생,파나진
391,2020-02-10 16:32,국순당,43650,내부결산시점 관리종목 지정 또는 상장폐지 사유 발생[정],국순당
392,2020-02-10 16:24,유아이디,69330,내부결산시점 관리종목 지정 또는 상장폐지 사유 발생,유아이디


In [27]:
#저장
bankrupt_df.to_csv('kind_bankrupt_df.csv')

### bankrupt_df 파일 불러온 후 시간 슬라이싱

In [4]:
bankrupt = pd.read_csv(r'C:\Users\user\Desktop\code\부실기업\kind_bankrupt_df.csv', index_col=0)

In [5]:
#년도 추출
bankrupt['년'] = bankrupt['시간'].str[:4]

In [6]:
bankrupt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 394 entries, 0 to 393
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시간      394 non-null    object
 1   회사명     394 non-null    object
 2   종목코드    394 non-null    int64 
 3   공시제목    394 non-null    object
 4   제출인     386 non-null    object
 5   년       394 non-null    object
dtypes: int64(1), object(5)
memory usage: 21.5+ KB


In [ ]:
#dataset.loc[(dataset['occyp_type'] == 'Nan') & (dataset['DAYS_EMPLOYED'] == 0), 'occyp_type'] = 'jobless'

In [7]:
print(train.shape)
print(bankrupt.shape)

(27166, 7)
(394, 6)


### train, bankrupt merge위한 key 만들기 : 종목코드 + 년 

In [11]:
bankrupt['key'] = bankrupt['종목코드'].astype(str) +  bankrupt['년']

In [12]:
train['key'] = train['거래소코드'].astype(str) + train['년+1'].astype(str)
train_full['key'] = train_full['거래소코드'].astype(str) + train_full['년+1'].astype(str)

In [14]:
merged = pd.merge(train_full, bankrupt, on = 'key', how = 'outer')
# merged = pd.merge(train_full, bankrupt, on = 'key', how = 'outer')

In [189]:
merged = merged.fillna('임시')

In [190]:
merged[merged['회사명_x']=='임시']

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EV/EBITDA,년_x,년+1,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
27357,임시,임시,임시,임시,임시,임시,임시,임시,임시,임시,...,임시,임시,임시,318002000,2000-12-20 16:17,ST&I,31800.0,회사정리절차종결결정,에스티앤아이,2000
27358,임시,임시,임시,임시,임시,임시,임시,임시,임시,임시,...,임시,임시,임시,318002000,2000-11-23 10:19,ST&I,31800.0,회사정리절차종결신청,에스티앤아이,2000
27359,임시,임시,임시,임시,임시,임시,임시,임시,임시,임시,...,임시,임시,임시,318002000,2000-03-28 00:00,ST&I,31800.0,회사정리절차개시결정(씨티아이반도체(주) (1)),임시,2000
27360,임시,임시,임시,임시,임시,임시,임시,임시,임시,임시,...,임시,임시,임시,318002000,2000-03-28 00:00,ST&I,31800.0,회사정리절차개시결정(씨티아이반도체(주) (2)),임시,2000
27361,임시,임시,임시,임시,임시,임시,임시,임시,임시,임시,...,임시,임시,임시,338502000,2000-08-07 15:44,지노시스템,33850.0,화의채무변제완료보고서제출,지노시스템,2000
27362,임시,임시,임시,임시,임시,임시,임시,임시,임시,임시,...,임시,임시,임시,333102000,2000-06-29 10:33,엠투엔,33310.0,화의채무변제완료보고서제출,엠투엔,2000
27363,임시,임시,임시,임시,임시,임시,임시,임시,임시,임시,...,임시,임시,임시,333102000,2000-01-31 00:00,엠투엔,33310.0,은행거래재개((주)우진산전),임시,2000
27364,임시,임시,임시,임시,임시,임시,임시,임시,임시,임시,...,임시,임시,임시,133102000,2000-03-31 00:00,아진산업,13310.0,회사정리절차개시결정(아진산업(주)),임시,2000
27365,임시,임시,임시,임시,임시,임시,임시,임시,임시,임시,...,임시,임시,임시,283002000,2000-03-31 00:00,에이치엘비,28300.0,은행거래재개(국제정공(주)),임시,2000
27366,임시,임시,임시,임시,임시,임시,임시,임시,임시,임시,...,임시,임시,임시,283002000,2000-02-09 00:00,에이치엘비,28300.0,회사정리절차개시결정(국제정공(주) (1)),임시,2000


In [228]:
merged.head()

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EV/EBITDA,년_x,년+1,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
0,(주)CMG제약,58820.0,2000/12,5.0,임시,UQ,28990.0,21169.0,10541.0,4330.0,...,임시,2000,2001.0,588202001,임시,임시,임시,임시,임시,임시
1,(주)CMG제약,58820.0,2001/12,5.0,임시,UQ,20908.0,16359.0,2307.0,1829.0,...,임시,2001,2002.0,588202002,임시,임시,임시,임시,임시,임시
2,(주)CMG제약,58820.0,2002/12,5.0,임시,UQ,15401.0,9880.0,1436.0,355.0,...,임시,2002,2003.0,588202003,임시,임시,임시,임시,임시,임시
3,(주)CMG제약,58820.0,2003/12,5.0,임시,UQ,11063.0,8534.0,1083.0,2.0,...,임시,2003,2004.0,588202004,임시,임시,임시,임시,임시,임시
4,(주)CMG제약,58820.0,2004/12,5.0,임시,UQ,5179.0,3893.0,445.0,11.0,...,임시,2004,2005.0,588202005,임시,임시,임시,임시,임시,임시


In [191]:
print('train에 반영되지않은 부도회사 명 : ','\n', merged[merged['회사명_x']=='임시']['회사명_y'].unique())
print('-------------------------------------------------------------------')
print('train에 반영되지않은 부도회사 코드 : ', '\n', merged[merged['소속코드']=='임시']['종목코드'].unique())
print('-------------------------------------------------------------------')
print('train에 반영되지않은 부도회사 계 : ', merged[merged['회사명_x']=='임시']['회사명_y'].nunique())

train에 반영되지않은 부도회사 명 :  
 ['ST&I' '지노시스템' '엠투엔' '아진산업' '에이치엘비' '엔케이바이오' 'GK파워' '디보스' '평안물산'
 '에이프로테크놀로지' '우양에이치씨' '럭슬' '이노와이즈']
-------------------------------------------------------------------
train에 반영되지않은 부도회사 코드 :  
 [31800.0 33850.0 33310.0 13310.0 28300.0 19260.0 54020.0 80140.0 37240.0
 45470.0 101970.0 33600.0 86250.0]
-------------------------------------------------------------------
train에 반영되지않은 부도회사 계 :  13


## train에 반영되지않은 부도기업 13개 공시제목 채워넣기<br>
<span style="color:yellow">처리한 내역</span><br>
-31800.0 	(주)에스티앤아이<br>
-33850.0    (주)지노시스템<br>
-54020.0    (주)지케이파워<br>
-37240.0    평안물산(주)<br>
-45470.0    (주)에이프로테크놀로지<br>
-101970.0   우양에이치씨(주)<br>
-33600.0    럭슬(주)<br>
-86250.0    (주)화신테크<br>

<span style="color:yellow">이미 매칭 완료된 내역 존재</span><br>
-28300.0    에이치엘비(주)	<br>
-19260.0    (주)엔케이바이오<br>
-80140.0    (주)디보스<br>

<span style="color:yellow">특이사항</span><br>
-33310.0    (주)엠투엔 00년에 채무변제완료 후 은행거래 재개 -> 부도기업 아님으로 판단<br>
-13310.0    아진산업(주) -> 회사정리절차개시결정 후 20년간 영업중. 부도기업인지 아닌지 고려해봐야함

### st&i 31800.0

In [192]:
merged.loc[merged['거래소코드']==31800.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EV/EBITDA,년_x,년+1,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
9016,(주)에스티앤아이,31800.0,2000/12,4.0,2009/07/01,UQ,39665.0,3256.0,2928.0,511.0,...,임시,2000,2001.0,318002001,임시,임시,임시,임시,임시,임시
9017,(주)에스티앤아이,31800.0,2001/12,4.0,2009/07/01,UQ,34205.0,2258.0,1407.0,61.0,...,임시,2001,2002.0,318002002,임시,임시,임시,임시,임시,임시
9018,(주)에스티앤아이,31800.0,2002/12,4.0,2009/07/01,UQ,67242.0,37320.0,12111.0,1054.0,...,임시,2002,2003.0,318002003,임시,임시,임시,임시,임시,임시
9019,(주)에스티앤아이,31800.0,2003/12,4.0,2009/07/01,UQ,51446.0,17927.0,10882.0,2569.0,...,임시,2003,2004.0,318002004,임시,임시,임시,임시,임시,임시
9020,(주)에스티앤아이,31800.0,2004/12,4.0,2009/07/01,UQ,42000.0,11661.0,10886.0,9555.0,...,임시,2004,2005.0,318002005,임시,임시,임시,임시,임시,임시
9021,(주)에스티앤아이,31800.0,2005/12,4.0,2009/07/01,UQ,34047.0,6314.0,4646.0,150.0,...,임시,2005,2006.0,318002006,임시,임시,임시,임시,임시,임시
9022,(주)에스티앤아이,31800.0,2006/12,4.0,2009/07/01,UQ,57281.0,16155.0,13271.0,1507.0,...,임시,2006,2007.0,318002007,임시,임시,임시,임시,임시,임시
9023,(주)에스티앤아이,31800.0,2007/12,4.0,2009/07/01,UQ,65077.0,26264.0,24626.0,204.0,...,0.0,2007,2008.0,318002008,임시,임시,임시,임시,임시,임시
9024,(주)에스티앤아이,31800.0,2008/12,4.0,2009/07/01,UQ,8326.0,1243.0,1243.0,36.0,...,임시,2008,2009.0,318002009,임시,임시,임시,임시,임시,임시


In [193]:
merged.loc[merged['종목코드']==31800.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EV/EBITDA,년_x,년+1,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
27357,임시,임시,임시,임시,임시,임시,임시,임시,임시,임시,...,임시,임시,임시,318002000,2000-12-20 16:17,ST&I,31800.0,회사정리절차종결결정,에스티앤아이,2000
27358,임시,임시,임시,임시,임시,임시,임시,임시,임시,임시,...,임시,임시,임시,318002000,2000-11-23 10:19,ST&I,31800.0,회사정리절차종결신청,에스티앤아이,2000
27359,임시,임시,임시,임시,임시,임시,임시,임시,임시,임시,...,임시,임시,임시,318002000,2000-03-28 00:00,ST&I,31800.0,회사정리절차개시결정(씨티아이반도체(주) (1)),임시,2000
27360,임시,임시,임시,임시,임시,임시,임시,임시,임시,임시,...,임시,임시,임시,318002000,2000-03-28 00:00,ST&I,31800.0,회사정리절차개시결정(씨티아이반도체(주) (2)),임시,2000


In [194]:
merged.loc[merged['key']=='318002000', '공시제목'] = '회사정리절차종결결정'

### **엠투엔** 33310.0 -> 부도기업 아닌걸로 판단 

In [195]:
merged.loc[merged['거래소코드']==33310.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EV/EBITDA,년_x,년+1,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
10414,(주)엠투엔,33310.0,2000/12,5.0,임시,QG,21993.0,9456.0,5107.0,4.0,...,임시,2000,2001.0,333102001,임시,임시,임시,임시,임시,임시
10415,(주)엠투엔,33310.0,2001/12,5.0,임시,UQ,23635.0,9988.0,5819.0,681.0,...,임시,2001,2002.0,333102002,임시,임시,임시,임시,임시,임시
10416,(주)엠투엔,33310.0,2002/12,5.0,임시,UQ,29815.0,15254.0,8024.0,383.0,...,임시,2002,2003.0,333102003,임시,임시,임시,임시,임시,임시
10417,(주)엠투엔,33310.0,2003/12,5.0,임시,UQ,23895.0,14267.0,6723.0,378.0,...,임시,2003,2004.0,333102004,임시,임시,임시,임시,임시,임시
10418,(주)엠투엔,33310.0,2004/12,5.0,임시,UQ,24814.0,14154.0,6537.0,1071.0,...,임시,2004,2005.0,333102005,임시,임시,임시,임시,임시,임시
10419,(주)엠투엔,33310.0,2005/12,5.0,임시,UQ,25800.0,14601.0,7325.0,997.0,...,임시,2005,2006.0,333102006,임시,임시,임시,임시,임시,임시
10420,(주)엠투엔,33310.0,2006/12,5.0,임시,UQ,28862.0,17819.0,9946.0,1343.0,...,임시,2006,2007.0,333102007,임시,임시,임시,임시,임시,임시
10421,(주)엠투엔,33310.0,2007/12,5.0,임시,UQ,61285.0,18852.0,13862.0,3770.0,...,86.41,2007,2008.0,333102008,임시,임시,임시,임시,임시,임시
10422,(주)엠투엔,33310.0,2008/12,5.0,임시,UQ,73832.0,24728.0,10373.0,91.0,...,9.46,2008,2009.0,333102009,임시,임시,임시,임시,임시,임시
10423,(주)엠투엔,33310.0,2009/12,5.0,임시,UQ,67139.0,21664.0,12380.0,2726.0,...,41.78,2009,2010.0,333102010,임시,임시,임시,임시,임시,임시


In [196]:
merged[merged['종목코드']==33310.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EV/EBITDA,년_x,년+1,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
27362,임시,임시,임시,임시,임시,임시,임시,임시,임시,임시,...,임시,임시,임시,333102000,2000-06-29 10:33,엠투엔,33310.0,화의채무변제완료보고서제출,엠투엔,2000
27363,임시,임시,임시,임시,임시,임시,임시,임시,임시,임시,...,임시,임시,임시,333102000,2000-01-31 00:00,엠투엔,33310.0,은행거래재개((주)우진산전),임시,2000


In [197]:
merged.loc[merged['key'] == '333102021', '공시제목'] = '화의채무변제완료보고서제출'

### 지노

In [198]:
merged.loc[merged['거래소코드'] == 33850.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EV/EBITDA,년_x,년+1,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
14214,(주)지노시스템,33850.0,2000/12,4.0,2011/05/07,QG,13655.0,8846.0,7485.0,1919.0,...,임시,2000,2001.0,338502001,임시,임시,임시,임시,임시,임시
14215,(주)지노시스템,33850.0,2001/12,4.0,2011/05/07,UQ,22179.0,15593.0,14762.0,2219.0,...,임시,2001,2002.0,338502002,임시,임시,임시,임시,임시,임시
14216,(주)지노시스템,33850.0,2002/12,4.0,2011/05/07,UQ,25466.0,17229.0,15134.0,2891.0,...,임시,2002,2003.0,338502003,임시,임시,임시,임시,임시,임시
14217,(주)지노시스템,33850.0,2003/12,4.0,2011/05/07,UQ,26862.0,17129.0,16437.0,1644.0,...,임시,2003,2004.0,338502004,임시,임시,임시,임시,임시,임시
14218,(주)지노시스템,33850.0,2004/12,4.0,2011/05/07,UQ,16319.0,8982.0,8494.0,41.0,...,임시,2004,2005.0,338502005,임시,임시,임시,임시,임시,임시
14219,(주)지노시스템,33850.0,2005/12,4.0,2011/05/07,UQ,19363.0,7509.0,7218.0,174.0,...,임시,2005,2006.0,338502006,임시,임시,임시,임시,임시,임시
14220,(주)지노시스템,33850.0,2006/12,4.0,2011/05/07,UQ,22226.0,13669.0,13502.0,5023.0,...,임시,2006,2007.0,338502007,임시,임시,임시,임시,임시,임시
14221,(주)지노시스템,33850.0,2007/12,4.0,2011/05/07,UQ,30430.0,17580.0,17538.0,4681.0,...,0.0,2007,2008.0,338502008,임시,임시,임시,임시,임시,임시
14222,(주)지노시스템,33850.0,2008/12,4.0,2011/05/07,UQ,26845.0,12301.0,10110.0,2261.0,...,임시,2008,2009.0,338502009,임시,임시,임시,임시,임시,임시
14223,(주)지노시스템,33850.0,2009/12,4.0,2011/05/07,UQ,20577.0,7062.0,6347.0,3359.0,...,임시,2009,2010.0,338502010,임시,임시,임시,임시,임시,임시


In [199]:
merged.loc[merged['종목코드'] == 33850.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EV/EBITDA,년_x,년+1,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
27361,임시,임시,임시,임시,임시,임시,임시,임시,임시,임시,...,임시,임시,임시,338502000,2000-08-07 15:44,지노시스템,33850.0,화의채무변제완료보고서제출,지노시스템,2000


In [200]:
merged.loc[merged['key'] == '338502011']

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EV/EBITDA,년_x,년+1,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
14224,(주)지노시스템,33850.0,2010/12,4.0,2011/05/07,DS,34650.0,23288.0,18166.0,365.0,...,임시,2010,2011.0,338502011,임시,임시,임시,임시,임시,임시


### **아진산업** 13310.0 -다시 보기 임시친구 -> 회사정리절차개시결정 후 20년간 영업중 고려해봐야함

In [201]:
merged.loc[merged['거래소코드'] == 13310.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EV/EBITDA,년_x,년+1,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
22951,아진산업(주),13310.0,2002/12,5.0,임시,UQ,22950.0,8013.0,7325.0,656.0,...,임시,2002,2003.0,133102003,임시,임시,임시,임시,임시,임시
22952,아진산업(주),13310.0,2003/12,5.0,임시,UQ,21161.0,7393.0,6661.0,9.0,...,임시,2003,2004.0,133102004,임시,임시,임시,임시,임시,임시
22953,아진산업(주),13310.0,2004/12,5.0,임시,UQ,20043.0,4902.0,4035.0,1665.0,...,임시,2004,2005.0,133102005,임시,임시,임시,임시,임시,임시
22954,아진산업(주),13310.0,2005/12,5.0,임시,UQ,34809.0,11729.0,10733.0,6520.0,...,임시,2005,2006.0,133102006,임시,임시,임시,임시,임시,임시
22955,아진산업(주),13310.0,2006/12,5.0,임시,UQ,37867.0,6117.0,4882.0,1.0,...,임시,2006,2007.0,133102007,임시,임시,임시,임시,임시,임시
22956,아진산업(주),13310.0,2007/12,5.0,임시,UQ,45183.0,8901.0,7359.0,466.0,...,1.45,2007,2008.0,133102008,임시,임시,임시,임시,임시,임시
22957,아진산업(주),13310.0,2008/12,5.0,임시,UQ,91325.0,17752.0,15222.0,272.0,...,3.74,2008,2009.0,133102009,임시,임시,임시,임시,임시,임시
22958,아진산업(주),13310.0,2009/12,5.0,임시,UQ,110853.0,32332.0,29509.0,246.0,...,3.48,2009,2010.0,133102010,임시,임시,임시,임시,임시,임시
22959,아진산업(주),13310.0,2010/12,5.0,임시,UQ,150757.0,54306.0,46940.0,5377.0,...,2.74,2010,2011.0,133102011,임시,임시,임시,임시,임시,임시
22960,아진산업(주),13310.0,2011/12,5.0,임시,UQ,187676.0,66664.0,64329.0,434.0,...,2.2,2011,2012.0,133102012,임시,임시,임시,임시,임시,임시


In [202]:
merged.loc[merged['종목코드'] == 13310.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EV/EBITDA,년_x,년+1,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
27364,임시,임시,임시,임시,임시,임시,임시,임시,임시,임시,...,임시,임시,임시,133102000,2000-03-31 00:00,아진산업,13310.0,회사정리절차개시결정(아진산업(주)),임시,2000


In [203]:
merged.loc[merged['key'] == 133102021, '공시제목'] = '회사정리절차개시결정(아진산업(주))'

In [204]:
merged.loc[merged['종목코드'] == 13310.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EV/EBITDA,년_x,년+1,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
27364,임시,임시,임시,임시,임시,임시,임시,임시,임시,임시,...,임시,임시,임시,133102000,2000-03-31 00:00,아진산업,13310.0,회사정리절차개시결정(아진산업(주)),임시,2000


### 에이치엘비 28300.0

In [205]:
merged.loc[merged['거래소코드'] == 28300.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EV/EBITDA,년_x,년+1,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
23576,에이치엘비(주),28300.0,2000/12,5.0,임시,UQ,22460.0,3362.0,1775.0,216.0,...,임시,2000,2001.0,283002001,임시,임시,임시,임시,임시,임시
23577,에이치엘비(주),28300.0,2001/12,5.0,임시,UQ,17974.0,2380.0,1243.0,3.0,...,임시,2001,2002.0,283002002,2002-05-21 18:16,에이치엘비,28300.0,회사정리절차종결결정,에이치엘비,2002
23578,에이치엘비(주),28300.0,2001/12,5.0,임시,UQ,17974.0,2380.0,1243.0,3.0,...,임시,2001,2002.0,283002002,2002-04-17 17:42,에이치엘비,28300.0,회사정리절차종결신청,에이치엘비,2002
23579,에이치엘비(주),28300.0,2002/12,5.0,임시,UQ,16407.0,2497.0,1990.0,30.0,...,임시,2002,2003.0,283002003,임시,임시,임시,임시,임시,임시
23580,에이치엘비(주),28300.0,2003/12,5.0,임시,QG,11762.0,726.0,645.0,303.0,...,임시,2003,2004.0,283002004,임시,임시,임시,임시,임시,임시
23581,에이치엘비(주),28300.0,2004/12,5.0,임시,UQ,2347.0,1768.0,1768.0,15.0,...,임시,2004,2005.0,283002005,임시,임시,임시,임시,임시,임시
23582,에이치엘비(주),28300.0,2005/12,5.0,임시,UQ,19044.0,7618.0,7318.0,3858.0,...,임시,2005,2006.0,283002006,임시,임시,임시,임시,임시,임시
23583,에이치엘비(주),28300.0,2006/12,5.0,임시,UQ,27051.0,15420.0,15063.0,7998.0,...,임시,2006,2007.0,283002007,임시,임시,임시,임시,임시,임시
23584,에이치엘비(주),28300.0,2007/12,5.0,임시,UQ,43462.0,27231.0,27209.0,13265.0,...,0.0,2007,2008.0,283002008,임시,임시,임시,임시,임시,임시
23585,에이치엘비(주),28300.0,2008/12,5.0,임시,UQ,50147.0,10165.0,9613.0,169.0,...,임시,2008,2009.0,283002009,임시,임시,임시,임시,임시,임시


### '엔케이바이오' 19260.0

In [206]:
merged.loc[merged['거래소코드'] == 19260.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EV/EBITDA,년_x,년+1,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
10017,(주)엔케이바이오,19260.0,2000/12,4.0,2012/09/05,UQ,31785.0,7360.0,5442.0,540.0,...,임시,2000,2001.0,192602001,2001-02-22 18:34,엔케이바이오,19260.0,화의채무변제완료보고서제출,셀텍,2001
10018,(주)엔케이바이오,19260.0,2001/12,4.0,2012/09/05,UQ,26915.0,5537.0,4495.0,719.0,...,임시,2001,2002.0,192602002,임시,임시,임시,임시,임시,임시
10019,(주)엔케이바이오,19260.0,2002/12,4.0,2012/09/05,UQ,24184.0,4948.0,3687.0,464.0,...,임시,2002,2003.0,192602003,임시,임시,임시,임시,임시,임시
10020,(주)엔케이바이오,19260.0,2003/12,4.0,2012/09/05,UQ,23685.0,7548.0,6099.0,1321.0,...,임시,2003,2004.0,192602004,임시,임시,임시,임시,임시,임시
10021,(주)엔케이바이오,19260.0,2004/12,4.0,2012/09/05,UQ,37651.0,10036.0,8051.0,1704.0,...,임시,2004,2005.0,192602005,임시,임시,임시,임시,임시,임시
10022,(주)엔케이바이오,19260.0,2005/12,4.0,2012/09/05,UQ,43491.0,12572.0,10307.0,3095.0,...,임시,2005,2006.0,192602006,임시,임시,임시,임시,임시,임시
10023,(주)엔케이바이오,19260.0,2006/12,4.0,2012/09/05,UQ,54911.0,20037.0,18054.0,7822.0,...,임시,2006,2007.0,192602007,임시,임시,임시,임시,임시,임시
10024,(주)엔케이바이오,19260.0,2007/12,4.0,2012/09/05,UQ,40157.0,10451.0,8221.0,1348.0,...,0.0,2007,2008.0,192602008,임시,임시,임시,임시,임시,임시
10025,(주)엔케이바이오,19260.0,2008/12,4.0,2012/09/05,UQ,48850.0,6408.0,6017.0,1430.0,...,0.0,2008,2009.0,192602009,임시,임시,임시,임시,임시,임시
10026,(주)엔케이바이오,19260.0,2009/12,4.0,2012/09/05,UQ,46809.0,6424.0,5890.0,892.0,...,0.0,2009,2010.0,192602010,임시,임시,임시,임시,임시,임시


### GK파워 54020.0

In [207]:
merged.loc[merged['거래소코드'] == 54020.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EV/EBITDA,년_x,년+1,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
14469,(주)지케이파워,54020.0,2001/12,4.0,2010/04/03,UQ,23774.0,15963.0,9485.0,2031.0,...,임시,2001,2002.0,540202002,임시,임시,임시,임시,임시,임시
14470,(주)지케이파워,54020.0,2002/12,4.0,2010/04/03,UQ,32779.0,14046.0,8254.0,3281.0,...,임시,2002,2003.0,540202003,임시,임시,임시,임시,임시,임시
14471,(주)지케이파워,54020.0,2003/12,4.0,2010/04/03,UQ,21193.0,8242.0,5079.0,1966.0,...,임시,2003,2004.0,540202004,임시,임시,임시,임시,임시,임시
14472,(주)지케이파워,54020.0,2004/12,4.0,2010/04/03,UQ,12669.0,4478.0,3210.0,177.0,...,임시,2004,2005.0,540202005,임시,임시,임시,임시,임시,임시
14473,(주)지케이파워,54020.0,2005/12,4.0,2010/04/03,UQ,10098.0,4202.0,2169.0,758.0,...,임시,2005,2006.0,540202006,임시,임시,임시,임시,임시,임시
14474,(주)지케이파워,54020.0,2006/12,4.0,2010/04/03,UQ,9923.0,3973.0,2704.0,507.0,...,임시,2006,2007.0,540202007,임시,임시,임시,임시,임시,임시
14475,(주)지케이파워,54020.0,2007/12,4.0,2010/04/03,UQ,8611.0,3311.0,2994.0,127.0,...,0.0,2007,2008.0,540202008,임시,임시,임시,임시,임시,임시
14476,(주)지케이파워,54020.0,2008/12,4.0,2010/04/03,UQ,23590.0,19464.0,19121.0,17.0,...,임시,2008,2009.0,540202009,임시,임시,임시,임시,임시,임시


In [208]:
merged.loc[merged['종목코드'] == 54020.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EV/EBITDA,년_x,년+1,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
27369,임시,임시,임시,임시,임시,임시,임시,임시,임시,임시,...,임시,임시,임시,540202010,2010-03-23 14:29,GK파워,54020.0,부도발생,지케이파워,2010


In [209]:
merged.loc[merged['key'] == '540202009', '공시제목'] = '부도발생'

### 디보스 80140.0

In [210]:
merged.loc[merged['거래소코드'] == 80140.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EV/EBITDA,년_x,년+1,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
2242,(주)디보스,80140.0,2002/12,4.0,2010/02/02,UQ,12254.0,7505.0,3407.0,1699.0,...,임시,2002,2003.0,801402003,임시,임시,임시,임시,임시,임시
2243,(주)디보스,80140.0,2003/12,4.0,2010/02/02,UQ,22113.0,14942.0,8272.0,544.0,...,임시,2003,2004.0,801402004,임시,임시,임시,임시,임시,임시
2244,(주)디보스,80140.0,2004/12,4.0,2010/02/02,UQ,39390.0,28852.0,21590.0,277.0,...,임시,2004,2005.0,801402005,임시,임시,임시,임시,임시,임시
2245,(주)디보스,80140.0,2005/12,4.0,2010/02/02,UQ,61574.0,40417.0,20690.0,78.0,...,임시,2005,2006.0,801402006,임시,임시,임시,임시,임시,임시
2246,(주)디보스,80140.0,2006/12,4.0,2010/02/02,UQ,74910.0,49939.0,27833.0,3809.0,...,임시,2006,2007.0,801402007,임시,임시,임시,임시,임시,임시
2247,(주)디보스,80140.0,2007/12,4.0,2010/02/02,UQ,85008.0,56026.0,34775.0,7846.0,...,0.0,2007,2008.0,801402008,임시,임시,임시,임시,임시,임시
2248,(주)디보스,80140.0,2008/12,4.0,2010/02/02,UQ,41196.0,11050.0,4103.0,45.0,...,임시,2008,2009.0,801402009,2009-08-11 17:24,디보스,80140.0,[정정]회생절차개시결정,디보스,2009
2249,(주)디보스,80140.0,2008/12,4.0,2010/02/02,UQ,41196.0,11050.0,4103.0,45.0,...,임시,2008,2009.0,801402009,2009-07-21 16:44,디보스,80140.0,[정정]회생절차개시결정[정],디보스,2009
2250,(주)디보스,80140.0,2008/12,4.0,2010/02/02,UQ,41196.0,11050.0,4103.0,45.0,...,임시,2008,2009.0,801402009,2009-06-17 15:12,디보스,80140.0,[정정]회생절차개시결정[정],디보스,2009
2251,(주)디보스,80140.0,2008/12,4.0,2010/02/02,UQ,41196.0,11050.0,4103.0,45.0,...,임시,2008,2009.0,801402009,2009-04-21 15:07,디보스,80140.0,회생절차개시결정[정],디보스,2009


### 평안물산 37240.0

In [211]:
merged.loc[merged['거래소코드'] == 37240.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EV/EBITDA,년_x,년+1,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
26333,평안물산(주),37240.0,2000/12,4.0,2012/06/01,UQ,24759.0,17714.0,15759.0,3316.0,...,임시,2000,2001.0,372402001,임시,임시,임시,임시,임시,임시
26334,평안물산(주),37240.0,2001/12,4.0,2012/06/01,UQ,48056.0,32116.0,28217.0,11055.0,...,임시,2001,2002.0,372402002,임시,임시,임시,임시,임시,임시
26335,평안물산(주),37240.0,2002/12,4.0,2012/06/01,UQ,44564.0,27632.0,24048.0,5945.0,...,임시,2002,2003.0,372402003,임시,임시,임시,임시,임시,임시
26336,평안물산(주),37240.0,2003/12,4.0,2012/06/01,UQ,32601.0,17553.0,14699.0,3084.0,...,임시,2003,2004.0,372402004,임시,임시,임시,임시,임시,임시
26337,평안물산(주),37240.0,2004/12,4.0,2012/06/01,UQ,24300.0,13351.0,9905.0,1712.0,...,임시,2004,2005.0,372402005,임시,임시,임시,임시,임시,임시
26338,평안물산(주),37240.0,2005/12,4.0,2012/06/01,UQ,17459.0,10569.0,8315.0,1475.0,...,임시,2005,2006.0,372402006,임시,임시,임시,임시,임시,임시
26339,평안물산(주),37240.0,2006/12,4.0,2012/06/01,UQ,27098.0,16208.0,15432.0,10401.0,...,임시,2006,2007.0,372402007,임시,임시,임시,임시,임시,임시
26340,평안물산(주),37240.0,2007/12,4.0,2012/06/01,UQ,48249.0,13399.0,12769.0,1745.0,...,0.0,2007,2008.0,372402008,임시,임시,임시,임시,임시,임시
26341,평안물산(주),37240.0,2008/12,4.0,2012/06/01,UQ,38425.0,8981.0,8524.0,1179.0,...,임시,2008,2009.0,372402009,임시,임시,임시,임시,임시,임시
26342,평안물산(주),37240.0,2009/12,4.0,2012/06/01,UQ,21776.0,6177.0,6082.0,1967.0,...,임시,2009,2010.0,372402010,임시,임시,임시,임시,임시,임시


In [212]:
merged.loc[merged['종목코드'] == 37240.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EV/EBITDA,년_x,년+1,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
27371,임시,임시,임시,임시,임시,임시,임시,임시,임시,임시,...,임시,임시,임시,372402012,2012-03-15 19:32,평안물산,37240.0,내부결산시점 관리종목 지정 또는 상장폐지 사유 발생,평안물산,2012


In [213]:
merged.loc[merged['key'] == '372402011', '공시제목'] = '내부결산시점 관리종목 지정 또는 상장폐지 사유 발생'

### 에이프로테크놀로지 45470.0

In [214]:
merged.loc[merged['거래소코드'] == 45470.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EV/EBITDA,년_x,년+1,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
9547,(주)에이프로테크놀로지,45470.0,2000/12,4.0,2012/04/24,UQ,8491.0,5988.0,5473.0,116.0,...,임시,2000,2001.0,454702001,임시,임시,임시,임시,임시,임시
9548,(주)에이프로테크놀로지,45470.0,2001/12,4.0,2012/04/24,UQ,16676.0,11116.0,10570.0,822.0,...,임시,2001,2002.0,454702002,임시,임시,임시,임시,임시,임시
9549,(주)에이프로테크놀로지,45470.0,2002/12,4.0,2012/04/24,UQ,15814.0,11017.0,10270.0,226.0,...,임시,2002,2003.0,454702003,임시,임시,임시,임시,임시,임시
9550,(주)에이프로테크놀로지,45470.0,2003/12,4.0,2012/04/24,UQ,17830.0,10329.0,9763.0,749.0,...,임시,2003,2004.0,454702004,임시,임시,임시,임시,임시,임시
9551,(주)에이프로테크놀로지,45470.0,2004/12,4.0,2012/04/24,UQ,19749.0,8032.0,7029.0,126.0,...,임시,2004,2005.0,454702005,임시,임시,임시,임시,임시,임시
9552,(주)에이프로테크놀로지,45470.0,2005/12,4.0,2012/04/24,UQ,25537.0,15367.0,14761.0,518.0,...,임시,2005,2006.0,454702006,임시,임시,임시,임시,임시,임시
9553,(주)에이프로테크놀로지,45470.0,2006/12,4.0,2012/04/24,UQ,56938.0,36080.0,36070.0,27.0,...,임시,2006,2007.0,454702007,임시,임시,임시,임시,임시,임시
9554,(주)에이프로테크놀로지,45470.0,2007/12,4.0,2012/04/24,UQ,18484.0,5836.0,5751.0,24.0,...,0.0,2007,2008.0,454702008,임시,임시,임시,임시,임시,임시
9555,(주)에이프로테크놀로지,45470.0,2008/12,4.0,2012/04/24,UQ,16201.0,3047.0,3001.0,545.0,...,임시,2008,2009.0,454702009,임시,임시,임시,임시,임시,임시
9556,(주)에이프로테크놀로지,45470.0,2009/12,4.0,2012/04/24,UQ,42009.0,19975.0,14589.0,36.0,...,16.17,2009,2010.0,454702010,임시,임시,임시,임시,임시,임시


In [215]:
merged.loc[merged['종목코드'] == 45470.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EV/EBITDA,년_x,년+1,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
27372,임시,임시,임시,임시,임시,임시,임시,임시,임시,임시,...,임시,임시,임시,454702012,2012-03-15 18:24,에이프로테크놀로지,45470.0,내부결산시점 관리종목 지정 또는 상장폐지 사유 발생,에이프로테크놀로지,2012


In [216]:
merged.loc[merged['key'] == '454702011', '공시제목'] = '내부결산시점 관리종목 지정 또는 상장폐지 사유 발생'

### 우양에이치씨 101970.0

In [217]:
merged.loc[merged['거래소코드'] == 101970.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EV/EBITDA,년_x,년+1,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
24513,우양에이치씨(주),101970.0,2003/12,4.0,2015/03/17,UQ,10168.0,5048.0,4268.0,453.0,...,임시,2003,2004.0,1019702004,임시,임시,임시,임시,임시,임시
24514,우양에이치씨(주),101970.0,2004/12,4.0,2015/03/17,UQ,24334.0,7796.0,6731.0,772.0,...,임시,2004,2005.0,1019702005,임시,임시,임시,임시,임시,임시
24515,우양에이치씨(주),101970.0,2005/12,4.0,2015/03/17,UQ,34892.0,13034.0,12722.0,177.0,...,임시,2005,2006.0,1019702006,임시,임시,임시,임시,임시,임시
24516,우양에이치씨(주),101970.0,2006/12,4.0,2015/03/17,UQ,66313.0,30264.0,25145.0,143.0,...,임시,2006,2007.0,1019702007,임시,임시,임시,임시,임시,임시
24517,우양에이치씨(주),101970.0,2007/12,4.0,2015/03/17,UQ,78402.0,35089.0,31054.0,443.0,...,2.53,2007,2008.0,1019702008,임시,임시,임시,임시,임시,임시
24518,우양에이치씨(주),101970.0,2008/12,4.0,2015/03/17,UQ,131395.0,77255.0,69711.0,1422.0,...,5.03,2008,2009.0,1019702009,임시,임시,임시,임시,임시,임시
24519,우양에이치씨(주),101970.0,2009/12,4.0,2015/03/17,UQ,161973.0,100978.0,97740.0,11569.0,...,1.95,2009,2010.0,1019702010,임시,임시,임시,임시,임시,임시
24520,우양에이치씨(주),101970.0,2010/12,4.0,2015/03/17,UQ,210294.0,133253.0,125697.0,1470.0,...,4.57,2010,2011.0,1019702011,임시,임시,임시,임시,임시,임시
24521,우양에이치씨(주),101970.0,2011/12,4.0,2015/03/17,UQ,233181.0,153383.0,143876.0,4510.0,...,3.29,2011,2012.0,1019702012,임시,임시,임시,임시,임시,임시
24522,우양에이치씨(주),101970.0,2012/12,4.0,2015/03/17,UQ,316015.0,193413.0,182311.0,6934.0,...,7.25,2012,2013.0,1019702013,임시,임시,임시,임시,임시,임시


In [218]:
merged.loc[merged['종목코드'] == 101970.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EV/EBITDA,년_x,년+1,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
27373,임시,임시,임시,임시,임시,임시,임시,임시,임시,임시,...,임시,임시,임시,1019702015,2015-03-04 16:49,우양에이치씨,101970.0,부도발생,우양에이치씨,2015
27374,임시,임시,임시,임시,임시,임시,임시,임시,임시,임시,...,임시,임시,임시,1019702015,2015-03-02 18:15,우양에이치씨,101970.0,회생절차개시신청,우양에이치씨,2015


In [219]:
merged.loc[merged['key'] == '1019702014', '공시제목'] = '부도발생'

### 럭슬 33600.0

In [220]:
merged.loc[merged['거래소코드'] == 33600.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EV/EBITDA,년_x,년+1,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
21173,럭슬(주),33600.0,2000/12,4.0,2021/08/11,UQ,39379.0,20589.0,20383.0,4228.0,...,임시,2000,2001.0,336002001,임시,임시,임시,임시,임시,임시
21174,럭슬(주),33600.0,2001/12,4.0,2021/08/11,UQ,42842.0,18792.0,18163.0,7707.0,...,임시,2001,2002.0,336002002,임시,임시,임시,임시,임시,임시
21175,럭슬(주),33600.0,2002/12,4.0,2021/08/11,UQ,50805.0,24888.0,23638.0,7598.0,...,임시,2002,2003.0,336002003,임시,임시,임시,임시,임시,임시
21176,럭슬(주),33600.0,2003/12,4.0,2021/08/11,UQ,49951.0,20707.0,18521.0,8826.0,...,임시,2003,2004.0,336002004,임시,임시,임시,임시,임시,임시
21177,럭슬(주),33600.0,2004/12,4.0,2021/08/11,UQ,46989.0,14891.0,13739.0,5160.0,...,임시,2004,2005.0,336002005,임시,임시,임시,임시,임시,임시
21178,럭슬(주),33600.0,2005/12,4.0,2021/08/11,UQ,58238.0,16699.0,15518.0,3626.0,...,임시,2005,2006.0,336002006,임시,임시,임시,임시,임시,임시
21179,럭슬(주),33600.0,2006/12,4.0,2021/08/11,UQ,56398.0,15533.0,13996.0,3704.0,...,임시,2006,2007.0,336002007,임시,임시,임시,임시,임시,임시
21180,럭슬(주),33600.0,2007/12,4.0,2021/08/11,UQ,54904.0,18249.0,17188.0,5363.0,...,6.43,2007,2008.0,336002008,임시,임시,임시,임시,임시,임시
21181,럭슬(주),33600.0,2008/12,4.0,2021/08/11,UQ,88812.0,26849.0,25896.0,6730.0,...,6.06,2008,2009.0,336002009,임시,임시,임시,임시,임시,임시
21182,럭슬(주),33600.0,2009/12,4.0,2021/08/11,UQ,91332.0,27214.0,25808.0,2950.0,...,27.21,2009,2010.0,336002010,임시,임시,임시,임시,임시,임시


In [221]:
merged.loc[merged['종목코드'] == 33600.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EV/EBITDA,년_x,년+1,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
27375,임시,임시,임시,임시,임시,임시,임시,임시,임시,임시,...,임시,임시,임시,336002020,2020-11-17 16:01,럭슬,33600.0,[정정]회생절차개시신청,럭슬,2020
27376,임시,임시,임시,임시,임시,임시,임시,임시,임시,임시,...,임시,임시,임시,336002020,2020-09-02 15:40,럭슬,33600.0,회생절차개시신청[정],럭슬,2020


In [222]:
merged.loc[merged['key'] == '336002019', '공시제목']='[정정]회생절차개시신청'

### 이노와이즈 86250.0

In [227]:
merged.loc[merged['거래소코드'] == 86250.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EV/EBITDA,년_x,년+1,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
19275,(주)화신테크,86250.0,2000/12,4.0,2021/08/31,UQ,15068.0,6276.0,6240.0,130.0,...,임시,2000,2001.0,862502001,임시,임시,임시,임시,임시,임시
19276,(주)화신테크,86250.0,2001/12,4.0,2021/08/31,UQ,14566.0,4417.0,4412.0,126.0,...,임시,2001,2002.0,862502002,임시,임시,임시,임시,임시,임시
19277,(주)화신테크,86250.0,2002/12,4.0,2021/08/31,UQ,13941.0,3788.0,3784.0,49.0,...,임시,2002,2003.0,862502003,임시,임시,임시,임시,임시,임시
19278,(주)화신테크,86250.0,2003/12,4.0,2021/08/31,UQ,16421.0,4539.0,4536.0,325.0,...,임시,2003,2004.0,862502004,임시,임시,임시,임시,임시,임시
19279,(주)화신테크,86250.0,2004/12,4.0,2021/08/31,UQ,19357.0,5994.0,5989.0,2179.0,...,임시,2004,2005.0,862502005,임시,임시,임시,임시,임시,임시
19280,(주)화신테크,86250.0,2005/12,4.0,2021/08/31,UQ,26217.0,7021.0,7017.0,642.0,...,임시,2005,2006.0,862502006,임시,임시,임시,임시,임시,임시
19281,(주)화신테크,86250.0,2006/12,4.0,2021/08/31,UQ,34097.0,14522.0,14518.0,1276.0,...,임시,2006,2007.0,862502007,임시,임시,임시,임시,임시,임시
19282,(주)화신테크,86250.0,2007/12,4.0,2021/08/31,UQ,33153.0,12018.0,12014.0,3118.0,...,0.0,2007,2008.0,862502008,임시,임시,임시,임시,임시,임시
19283,(주)화신테크,86250.0,2008/12,4.0,2021/08/31,UQ,35176.0,11011.0,10998.0,654.0,...,1.09,2008,2009.0,862502009,임시,임시,임시,임시,임시,임시
19284,(주)화신테크,86250.0,2009/12,4.0,2021/08/31,UQ,37843.0,7507.0,7499.0,1003.0,...,14.67,2009,2010.0,862502010,임시,임시,임시,임시,임시,임시


In [224]:
merged.loc[merged['종목코드'] == 86250.0]

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EV/EBITDA,년_x,년+1,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
27377,임시,임시,임시,임시,임시,임시,임시,임시,임시,임시,...,임시,임시,임시,862502020,2020-07-28 14:54,이노와이즈,86250.0,회생절차개시결정,화신테크,2020
27378,임시,임시,임시,임시,임시,임시,임시,임시,임시,임시,...,임시,임시,임시,862502020,2020-05-08 15:03,이노와이즈,86250.0,회생절차개시신청,화신테크,2020


In [225]:
merged.loc[merged['key'] == '862502021', '공시제목'] = '회생절차개시신청'

# 하다가말았다

In [226]:
merged

,회사명_x,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,자산총계(요약),유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),...,EV/EBITDA,년_x,년+1,key,시간,회사명_y,종목코드,공시제목,제출인,년_y
0,(주)CMG제약,58820.0,2000/12,5.0,임시,UQ,28990.0,21169.0,10541.0,4330.0,...,임시,2000,2001.0,588202001,임시,임시,임시,임시,임시,임시
1,(주)CMG제약,58820.0,2001/12,5.0,임시,UQ,20908.0,16359.0,2307.0,1829.0,...,임시,2001,2002.0,588202002,임시,임시,임시,임시,임시,임시
2,(주)CMG제약,58820.0,2002/12,5.0,임시,UQ,15401.0,9880.0,1436.0,355.0,...,임시,2002,2003.0,588202003,임시,임시,임시,임시,임시,임시
3,(주)CMG제약,58820.0,2003/12,5.0,임시,UQ,11063.0,8534.0,1083.0,2.0,...,임시,2003,2004.0,588202004,임시,임시,임시,임시,임시,임시
4,(주)CMG제약,58820.0,2004/12,5.0,임시,UQ,5179.0,3893.0,445.0,11.0,...,임시,2004,2005.0,588202005,임시,임시,임시,임시,임시,임시
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27374,임시,임시,임시,임시,임시,임시,임시,임시,임시,임시,...,임시,임시,임시,1019702015,2015-03-02 18:15,우양에이치씨,101970.0,회생절차개시신청,우양에이치씨,2015
27375,임시,임시,임시,임시,임시,임시,임시,임시,임시,임시,...,임시,임시,임시,336002020,2020-11-17 16:01,럭슬,33600.0,[정정]회생절차개시신청,럭슬,2020
27376,임시,임시,임시,임시,임시,임시,임시,임시,임시,임시,...,임시,임시,임시,336002020,2020-09-02 15:40,럭슬,33600.0,회생절차개시신청[정],럭슬,2020
27377,임시,임시,임시,임시,임시,임시,임시,임시,임시,임시,...,임시,임시,임시,862502020,2020-07-28 14:54,이노와이즈,86250.0,회생절차개시결정,화신테크,2020


In [22]:
bankrupt[bankrupt['회사명']=='코썬바이오']

,시간,회사명,종목코드,공시제목,제출인,년도,key
374,2020-02-27 18:26,코썬바이오,204990,내부결산시점 관리종목 지정 또는 상장폐지 사유 발생,코썬바이오,2020,2049902020


In [20]:
train[train['회사명']=='(주)코썬바이오']

,회사명,거래소코드,회계년도,총자본증가율,유형자산증가율,유동자산증가율,영업이익증가율,경상이익증가율 (2007년 이전 발생),순이익증가율,재고자산증가율,...,기타포괄손익누계액(요약),이익잉여금(요약),*미처분이익잉여금 및 차기이월미처분이익잉여금(요약),*당기순이익(요약),부채와자본총계(요약),소속코드,상장폐지일,감사의견코드,년도,key
15726,(주)코썬바이오,204990,2014/12,26.85,-4.75,38.84,160.29,0.0,274.17,-17.57,...,NaN,NaN,NaN,NaN,NaN,6.0,NaN,UQ,2014,2049902014
15727,(주)코썬바이오,204990,2015/12,46.15,23.24,59.65,29.38,0.0,38.30,295.73,...,NaN,NaN,NaN,NaN,NaN,6.0,NaN,UQ,2015,2049902015
15728,(주)코썬바이오,204990,2019/12,-84.22,-18.22,-87.25,0.00,NaN,0.00,-48.50,...,NaN,NaN,NaN,NaN,NaN,6.0,NaN,UQ,2019,2049902019
15729,(주)코썬바이오,204990,2016/12,56.74,1.33,73.30,-17.59,0.0,-15.40,-13.14,...,NaN,NaN,NaN,NaN,NaN,6.0,NaN,UQ,2016,2049902016
15730,(주)코썬바이오,204990,2013/12,12.50,24.26,10.23,264.23,0.0,187.30,-9.95,...,NaN,NaN,NaN,NaN,NaN,6.0,NaN,UQ,2013,2049902013
15731,(주)코썬바이오,204990,2012/12,32.11,-7.86,56.93,0.00,0.0,0.00,-53.46,...,NaN,NaN,NaN,NaN,NaN,6.0,NaN,UQ,2012,2049902012
15732,(주)코썬바이오,204990,2018/12,-41.31,-5.54,-37.35,0.00,NaN,0.00,-17.52,...,NaN,NaN,NaN,NaN,NaN,6.0,NaN,UQ,2018,2049902018
15733,(주)코썬바이오,204990,2017/12,138.33,2.25,-8.60,0.00,NaN,0.00,17.23,...,NaN,NaN,NaN,NaN,NaN,6.0,NaN,UQ,2017,2049902017
15734,(주)코썬바이오,204990,2011/12,0.00,0.00,0.00,0.00,0.0,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,6.0,NaN,QS,2011,2049902011
15735,(주)코썬바이오,204990,2010/12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,2010,2049902010


In [15]:
merge[merge['거래소코드']==204990]

,회사명_x,거래소코드,회계년도,총자본증가율,유형자산증가율,유동자산증가율,영업이익증가율,경상이익증가율 (2007년 이전 발생),순이익증가율,재고자산증가율,...,상장폐지일,감사의견코드,년도_x,key,시간,회사명_y,종목코드,공시제목,제출인,년도_y
15802,(주)코썬바이오,204990.0,2014/12,26.85,-4.75,38.84,160.29,0.0,274.17,-17.57,...,NaN,UQ,2014,2049902014,NaN,NaN,NaN,NaN,NaN,NaN
15803,(주)코썬바이오,204990.0,2015/12,46.15,23.24,59.65,29.38,0.0,38.30,295.73,...,NaN,UQ,2015,2049902015,NaN,NaN,NaN,NaN,NaN,NaN
15804,(주)코썬바이오,204990.0,2019/12,-84.22,-18.22,-87.25,0.00,NaN,0.00,-48.50,...,NaN,UQ,2019,2049902019,NaN,NaN,NaN,NaN,NaN,NaN
15805,(주)코썬바이오,204990.0,2016/12,56.74,1.33,73.30,-17.59,0.0,-15.40,-13.14,...,NaN,UQ,2016,2049902016,NaN,NaN,NaN,NaN,NaN,NaN
15806,(주)코썬바이오,204990.0,2013/12,12.50,24.26,10.23,264.23,0.0,187.30,-9.95,...,NaN,UQ,2013,2049902013,NaN,NaN,NaN,NaN,NaN,NaN
15807,(주)코썬바이오,204990.0,2012/12,32.11,-7.86,56.93,0.00,0.0,0.00,-53.46,...,NaN,UQ,2012,2049902012,NaN,NaN,NaN,NaN,NaN,NaN
15808,(주)코썬바이오,204990.0,2018/12,-41.31,-5.54,-37.35,0.00,NaN,0.00,-17.52,...,NaN,UQ,2018,2049902018,NaN,NaN,NaN,NaN,NaN,NaN
15809,(주)코썬바이오,204990.0,2017/12,138.33,2.25,-8.60,0.00,NaN,0.00,17.23,...,NaN,UQ,2017,2049902017,NaN,NaN,NaN,NaN,NaN,NaN
15810,(주)코썬바이오,204990.0,2011/12,0.00,0.00,0.00,0.00,0.0,0.00,0.00,...,NaN,QS,2011,2049902011,NaN,NaN,NaN,NaN,NaN,NaN
15811,(주)코썬바이오,204990.0,2010/12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2010,2049902010,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
merge = pd.merge(train, bankrupt, on='key', how='outer')

In [16]:
merge

,회사명_x,거래소코드,회계년도,총자본증가율,유형자산증가율,유동자산증가율,영업이익증가율,경상이익증가율 (2007년 이전 발생),순이익증가율,재고자산증가율,...,상장폐지일,감사의견코드,년도_x,key,시간,회사명_y,종목코드,공시제목,제출인,년도_y
0,(주)CMG제약,58820.0,2000/12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,UQ,2000,588202000,NaN,NaN,NaN,NaN,NaN,NaN
1,(주)CMG제약,58820.0,2001/12,0.00,0.00,0.00,0.0,0.00,0.0,0.00,...,NaN,UQ,2001,588202001,NaN,NaN,NaN,NaN,NaN,NaN
2,(주)CMG제약,58820.0,2002/12,-26.34,-17.72,-39.61,0.0,NaN,0.0,-39.91,...,NaN,UQ,2002,588202002,NaN,NaN,NaN,NaN,NaN,NaN
3,(주)CMG제약,58820.0,2003/12,-28.17,-79.37,-13.62,0.0,0.00,0.0,-11.75,...,NaN,UQ,2003,588202003,NaN,NaN,NaN,NaN,NaN,NaN
4,(주)CMG제약,58820.0,2004/12,-53.18,-25.94,-54.38,0.0,-2128.48,0.0,-53.72,...,NaN,UQ,2004,588202004,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27327,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1972102020,2020-02-26 18:15,리드,197210.0,내부결산시점 관리종목 지정 또는 상장폐지 사유 발생[정],리드,2020
27328,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,302702020,2020-02-28 17:29,에스마크,30270.0,[정정]내부결산시점 관리종목 지정 또는 상장폐지 사유 발생,에스마크,2020
27329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,302702020,2020-02-14 17:18,에스마크,30270.0,내부결산시점 관리종목 지정 또는 상장폐지 사유 발생[정],에스마크,2020
27330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2049902020,2020-02-27 18:26,코썬바이오,204990.0,내부결산시점 관리종목 지정 또는 상장폐지 사유 발생,코썬바이오,2020


In [121]:
train.merge(bankrupt, on='key', join='inner')

TypeError: merge() got an unexpected keyword argument 'join'

In [104]:
train.loc[train['년도']=='2000']

,회사명,거래소코드,회계년도,총자본증가율,유형자산증가율,유동자산증가율,영업이익증가율,경상이익증가율 (2007년 이전 발생),순이익증가율,재고자산증가율,...,자본조정(요약),기타포괄손익누계액(요약),이익잉여금(요약),*미처분이익잉여금 및 차기이월미처분이익잉여금(요약),*당기순이익(요약),부채와자본총계(요약),소속코드,상장폐지일,감사의견코드,년도
0,(주)CMG제약,58820,2000/12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,UQ,2000
68,(주)강원,114190,2000/11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,UQ,2000
89,(주)강원랜드,35250,2000/12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,2003/09/04,UQ,2000
109,(주)경윤하이드로에너지,19120,2000/12,-13.62,2.78,-13.06,-50.19,-41.36,-39.12,-2.02,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,2011/04/06,UQ,2000
145,(주)골드앤에스,35290,2000/12,203.61,589.63,152.32,0.00,NaN,0.00,-48.31,...,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,DU,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26912,현주컴퓨터(주),38960,2000/06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,2005/05/11,UQ,2000
26927,현진소재(주),53660,2000/12,27.82,57.45,3.74,171.81,141.58,156.33,12.25,...,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,UQ,2000
26954,화성밸브(주),39610,2000/03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,UQ,2000
27001,후야인포넷(주),32050,2000/12,88.94,25.16,129.54,-53.76,-86.63,-83.36,95.59,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,2005/04/26,UQ,2000


In [1]:
pd.merge(train, bankrupt, how='outer')

NameError: name 'pd' is not defined

In [106]:
train.loc[(train['거래소코드'].astype(str) + '&' + train['년도']) == (bankrupt['종목코드'].astype(str) + '&' + bankrupt['년도'])]

ValueError: Can only compare identically-labeled Series objects

In [107]:
train['거래소코드'].astype(str) + ' & ' + train['년도']

0        58820 & 2000
1        58820 & 2001
2        58820 & 2002
3        58820 & 2003
4        58820 & 2004
             ...     
27054    24060 & 2004
27055    24060 & 2006
27056    24060 & 2002
27057    24060 & 2001
27058    24060 & 2003
Length: 27059, dtype: object

In [111]:
bankrupt['종목코드'].astype(str) + ' & ' + bankrupt['년도']

0       22220 & 2002
1       21040 & 2002
2       28300 & 2002
3       28300 & 2002
4       21040 & 2001
           ...      
389    103230 & 2020
390     46210 & 2020
391     43650 & 2020
392     69330 & 2020
393     88390 & 2020
Length: 394, dtype: object

In [112]:
for i in range(len(train)):
    if train['거래소코드'].astype(str) + ' & ' + train['년도'] == bankrupt['종목코드'].astype(str) + ' & ' + bankrupt['년도']: 
        train['부도사유'] = bankrupt['공시제목']
        
    else:
        pass


ValueError: Can only compare identically-labeled Series objects

In [ ]:
bankrupt

,시간,회사명,종목코드,공시제목,제출인,년도
0,2002-10-31 18:27,정산애강,22220,부도발생및은행거래정지,정산애강,2002
1,2002-10-08 12:06,대호특수강,21040,은행거래재개,대호특수강,2002
2,2002-05-21 18:16,에이치엘비,28300,회사정리절차종결결정,에이치엘비,2002
3,2002-04-17 17:42,에이치엘비,28300,회사정리절차종결신청,에이치엘비,2002
4,2001-12-26 14:58,대호특수강,21040,회사정리절차개시결정,대호특수강,2001
...,...,...,...,...,...,...
389,2020-02-12 16:21,에스앤더블류,103230,내부결산시점 관리종목 지정 또는 상장폐지 사유 발생,에스앤더블류,2020
390,2020-02-11 17:02,파나진,46210,내부결산시점 관리종목 지정 또는 상장폐지 사유 발생,파나진,2020
391,2020-02-10 16:32,국순당,43650,내부결산시점 관리종목 지정 또는 상장폐지 사유 발생[정],국순당,2020
392,2020-02-10 16:24,유아이디,69330,내부결산시점 관리종목 지정 또는 상장폐지 사유 발생,유아이디,2020


In [ ]:
train

,회사명,거래소코드,회계년도,총자본증가율,유형자산증가율,유동자산증가율,영업이익증가율,경상이익증가율 (2007년 이전 발생),순이익증가율,재고자산증가율,...,자본조정(요약),기타포괄손익누계액(요약),이익잉여금(요약),*미처분이익잉여금 및 차기이월미처분이익잉여금(요약),*당기순이익(요약),부채와자본총계(요약),소속코드,상장폐지일,감사의견코드,년도
0,(주)CMG제약,58820,2000/12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,UQ,2000
1,(주)CMG제약,58820,2001/12,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,UQ,2001
2,(주)CMG제약,58820,2002/12,-26.34,-17.72,-39.61,0.00,NaN,0.00,-39.91,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,UQ,2002
3,(주)CMG제약,58820,2003/12,-28.17,-79.37,-13.62,0.00,0.00,0.00,-11.75,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,UQ,2003
4,(주)CMG제약,58820,2004/12,-53.18,-25.94,-54.38,0.00,-2128.48,0.00,-53.72,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,UQ,2004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27054,흥구석유(주),24060,2004/12,0.21,0.07,-2.43,26.81,14.93,9.72,-61.31,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,UQ,2004
27055,흥구석유(주),24060,2006/12,1.60,1.69,24.04,-22.09,-15.73,-14.52,-18.56,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,UQ,2006
27056,흥구석유(주),24060,2002/12,4.71,-1.19,49.44,2.21,3.97,8.01,18.63,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,UQ,2002
27057,흥구석유(주),24060,2001/12,-8.26,-0.72,-33.74,2973.17,41.09,35.81,8.59,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,UQ,2001


In [ ]:
train.loc[(train['거래소코드'].astype(str)) & (train['년도']) == (bankrupt['종목코드'].astype(str)) & (bankrupt['년도']), ['감사의견']] = train['감사의견코드']

TypeError: unsupported operand type(s) for &: 'str' and 'str'

In [ ]:
train

,회사명,거래소코드,회계년도,총자본증가율,유형자산증가율,유동자산증가율,영업이익증가율,경상이익증가율 (2007년 이전 발생),순이익증가율,재고자산증가율,...,자본조정(요약),기타포괄손익누계액(요약),이익잉여금(요약),*미처분이익잉여금 및 차기이월미처분이익잉여금(요약),*당기순이익(요약),부채와자본총계(요약),소속코드,상장폐지일,감사의견코드,년도
0,(주)CMG제약,58820,2000/12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,UQ,2000
1,(주)CMG제약,58820,2001/12,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,UQ,2001
2,(주)CMG제약,58820,2002/12,-26.34,-17.72,-39.61,0.00,NaN,0.00,-39.91,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,UQ,2002
3,(주)CMG제약,58820,2003/12,-28.17,-79.37,-13.62,0.00,0.00,0.00,-11.75,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,UQ,2003
4,(주)CMG제약,58820,2004/12,-53.18,-25.94,-54.38,0.00,-2128.48,0.00,-53.72,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,UQ,2004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27054,흥구석유(주),24060,2004/12,0.21,0.07,-2.43,26.81,14.93,9.72,-61.31,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,UQ,2004
27055,흥구석유(주),24060,2006/12,1.60,1.69,24.04,-22.09,-15.73,-14.52,-18.56,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,UQ,2006
27056,흥구석유(주),24060,2002/12,4.71,-1.19,49.44,2.21,3.97,8.01,18.63,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,UQ,2002
27057,흥구석유(주),24060,2001/12,-8.26,-0.72,-33.74,2973.17,41.09,35.81,8.59,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,UQ,2001


In [ ]:
train['거래소코드'].dtype

dtype('int64')

In [ ]:
print(type(str(train['거래소코드'])))

<class 'str'>


In [ ]:
train['거래소코드']

0        58820
1        58820
2        58820
3        58820
4        58820
         ...  
27054    24060
27055    24060
27056    24060
27057    24060
27058    24060
Name: 거래소코드, Length: 27059, dtype: int64

In [ ]:
str(train['거래소코드'])

'0        58820\n1        58820\n2        58820\n3        58820\n4        58820\n         ...  \n27054    24060\n27055    24060\n27056    24060\n27057    24060\n27058    24060\nName: 거래소코드, Length: 27059, dtype: int64'

In [ ]:
train['거래소코드'].zfill(6)

AttributeError: 'Series' object has no attribute 'zfill'

In [ ]:
!pip install openpyxl

You should consider upgrading via the 'c:\users\user\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [ ]:
import xlrd
budo = xlrd.open_workbook(r'C:\apps\Ubion-CorporateBankruptcyAnalysis\Data\부도사유데이터\부도사유00~02.xls')

XLRDError: Unsupported format, or corrupt file: Expected BOF record; found b'\r\n\r\n\r\n\r\n'

In [ ]:
budo = pd.read_excel(r'C:\apps\Ubion-CorporateBankruptcyAnalysis\Data\부도사유데이터\부도사유00~02.xls')

ValueError: Excel file format cannot be determined, you must specify an engine manually.

In [ ]:
train

,회사명,거래소코드,회계년도,소속코드,상장폐지일,감사의견코드,총자본증가율,유형자산증가율,유동자산증가율,영업이익증가율,...,이연부채(요약),자본총계(요약),자본금(요약),자본잉여금(요약),자본조정(요약),기타포괄손익누계액(요약),이익잉여금(요약),*미처분이익잉여금 및 차기이월미처분이익잉여금(요약),*당기순이익(요약),부채와자본총계(요약)
0,(주)CMG제약,58820,2000/12,5.0,NaN,UQ,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,(주)CMG제약,58820,2001/12,5.0,NaN,UQ,0.00,0.00,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,(주)CMG제약,58820,2002/12,5.0,NaN,UQ,-26.34,-17.72,-39.61,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,(주)CMG제약,58820,2003/12,5.0,NaN,UQ,-28.17,-79.37,-13.62,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,(주)CMG제약,58820,2004/12,5.0,NaN,UQ,-53.18,-25.94,-54.38,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27054,흥구석유(주),24060,2004/12,5.0,NaN,UQ,0.21,0.07,-2.43,26.81,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27055,흥구석유(주),24060,2006/12,5.0,NaN,UQ,1.60,1.69,24.04,-22.09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27056,흥구석유(주),24060,2002/12,5.0,NaN,UQ,4.71,-1.19,49.44,2.21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27057,흥구석유(주),24060,2001/12,5.0,NaN,UQ,-8.26,-0.72,-33.74,2973.17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
train['거래소코드'].value_counts().sort_index()

250       21
380        4
440       21
1000      21
1540      21
          ..
950170     4
950180     3
950190     3
950200     2
950220     2
Name: 거래소코드, Length: 2058, dtype: int64

In [ ]:
train[train['회사명']== '(주)CMG제약']

,회사명,거래소코드,회계년도,유동자산(요약),당좌자산(요약),현금 및 현금성자산(요약),단기투자자산(요약),매출채권(요약),기타당좌자산(요약),비유동자산(요약),...,비유동부채(요약),사채(요약),장기차입금(요약),기타비유동부채(요약),자본총계(요약),자본금(요약),자본잉여금(요약),자본조정(요약),기타포괄손익누계액(요약),이익잉여금(요약)
0,(주)CMG제약,58820,2000/12,21169.0,10541.0,4330.0,4552.0,786.0,873.0,7821.0,...,0.0,5482.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,(주)CMG제약,58820,2001/12,16359.0,2307.0,1829.0,18.0,14.0,446.0,4549.0,...,0.0,110.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,(주)CMG제약,58820,2002/12,9880.0,1436.0,355.0,246.0,26.0,809.0,5522.0,...,0.0,-2144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,(주)CMG제약,58820,2003/12,8534.0,1083.0,2.0,381.0,72.0,628.0,2529.0,...,0.0,-2246.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,(주)CMG제약,58820,2004/12,3893.0,445.0,11.0,0.0,33.0,400.0,1286.0,...,0.0,-6119.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,(주)CMG제약,58820,2005/12,3449.0,3226.0,609.0,1500.0,264.0,853.0,4831.0,...,0.0,-8677.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,(주)CMG제약,58820,2006/12,6897.0,6897.0,2850.0,0.0,2400.0,1647.0,3133.0,...,0.0,-12088.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,(주)CMG제약,58820,2007/12,8952.0,8638.0,2714.0,3234.0,2122.0,568.0,13122.0,...,0.0,-12978.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,(주)CMG제약,58820,2008/12,16725.0,15015.0,6233.0,926.0,7267.0,590.0,17832.0,...,0.0,-18256.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,(주)CMG제약,58820,2009/12,9827.0,7398.0,1063.0,1105.0,4351.0,879.0,13008.0,...,0.0,-37358.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
for